In [59]:
import numpy as np
from tensorflow.contrib.keras import models, layers

In [60]:
vk_ids = ['alexlun', 'dmittov', 'nikita.zadorin', 'rudopysov', 'vanojima', 'ogion', 'ksv73']

dictionary = list({symbol for vk_id in vk_ids for symbol in vk_id})
sym2num = {symbol: idx for idx, symbol in enumerate(dictionary)}
symbols_count = len(dictionary)

def vectorize(vk_id):
    return np.array([sym2num[symbol] for symbol in vk_id]).astype(np.float32)

In [61]:
SZ = np.random.randint(10, 17)
input_layer = layers.Input(shape=(None,), name='InputLayer')
vk_id_embedding = layers.Embedding(input_dim=symbols_count, 
                                   output_dim=SZ, 
                                   name='Embedding')(input_layer)
rnn_grinder1 = layers.GRU(SZ, name='GRU', return_sequences=True)(vk_id_embedding)
rnn_grinder2 = layers.LSTM(SZ, name='LSTM')(rnn_grinder1)

dense1 = layers.Dense(SZ, activation='elu')(rnn_grinder2)
dropout = layers.Dropout(rate=.5)(dense1)
dense2 = layers.Dense(SZ, activation='elu')(dropout)
output = layers.Dense(1, activation='sigmoid')(dense2)

model = models.Model(inputs=input_layer, outputs=output)

In [62]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      (None, None)              0         
_________________________________________________________________
Embedding (Embedding)        (None, None, 14)          322       
_________________________________________________________________
GRU (GRU)                    (None, None, 14)          1218      
_________________________________________________________________
LSTM (LSTM)                  (None, 14)                1624      
_________________________________________________________________
dense_9 (Dense)              (None, 14)                210       
_________________________________________________________________
dropout_3 (Dropout)          (None, 14)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 14)                210       
__________

In [63]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [64]:
get_score = lambda vk_id: model.predict(vectorize(vk_id)[np.newaxis])[0][0]

In [65]:
scores_vk_ids = zip(vk_ids, [get_score(vk_id) for vk_id in vk_ids])
sorted_vk_ids_by_score = sorted(scores_vk_ids, key=lambda item: -item[1])
sorted_vk_ids_by_score[0]

('ogion', 0.48993602)

In [66]:
scores_vk_ids

[('alexlun', 0.46793318),
 ('dmittov', 0.48569021),
 ('nikita.zadorin', 0.47153062),
 ('rudopysov', 0.48122996),
 ('vanojima', 0.47534677),
 ('ogion', 0.48993602),
 ('ksv73', 0.47680512)]